In [12]:
#!pip install geopy
#!pip install osmnx
#!pip install geopandas
#!pip install pandas
#!pip install openpyxl

In [2]:
#!pip install osmnx
import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Creating Hospital Data

In [11]:
#Einlesen der Exceldatei
hospitals_df=pd.read_excel('data/hospital_data_3.xlsx') 

In [12]:
#auf sächsische Einrichtungen filtern
hospitals_df= hospitals_df[hospitals_df['Land'] == 14].reset_index(drop=True)
#Entfernung von nicht Versogungsrelevanten Einträgen
hospitals_df= hospitals_df[hospitals_df['INSG'] > 0].reset_index(drop=True)

In [13]:
# Einheitliche Konvertierung zu String und Entfernen unsichtbarer Zeichen
hospitals_df.columns = hospitals_df.columns.astype(str)
hospitals_df.columns = hospitals_df.columns.str.strip()

#Auswählen und umbenennen der relevanten Spalten
new_columns = {
	'Land':'Land',
	'Kreis':'Kreis',
	'Gemeinde':'Gemeinde',
	'Adresse_Name':'Name_Einrichtung',
	'Adresse_Strasse_Standort':'Adresse_Strasse_Standort',
	'Adresse_Haus-Nr._Standort':'Adresse_Haus-Nr._Standort',
	'Adresse_Ort_Standort':'Adresse_Ort_Standort',
    'Traeger':'Traeger',
	'EinrichtungsTyp':'EinrichtungsTyp',
	'Allgemeine_Notfallversorgung':'Allgemeine_Notfallversorgung',
	'INSG':'INSG',
	'0100':'Betten innere Medizin',
    '0300':'Betten Kardiologie ',
    '1500':'Betten Allgemeine Chirurgie',
	'2100':'Betten Herzchirurgie',
	'2200':'Betten Urologie',
	'2600':'Betten HNO',
	'2900':'Betten Psychatrie'
}
hospitals_df = hospitals_df[list(new_columns.keys())].rename(columns=new_columns)

In [14]:
#Hinzufügen der Adresse bestehend aus 3 Spalten
hospitals_df['Adresse'] = hospitals_df.apply(lambda row: ' '.join(map(str, row[['Adresse_Strasse_Standort', 'Adresse_Haus-Nr._Standort', 'Adresse_Ort_Standort']])), axis=1)

# Alte Spalten löschen
hospitals_df.drop(columns=['Adresse_Strasse_Standort', 'Adresse_Haus-Nr._Standort', 'Adresse_Ort_Standort'], inplace=True)

In [15]:
#Display(hospitals_df)

In [16]:
# Iteriere über jede Adresse und füge die Geo-Koordinaten ein
for index, row in hospitals_df.iterrows():
    adresse = row['Adresse']
    try:
        location_point = ox.geocode(adresse)
        point = Point(location_point[1], location_point[0])  # Point(longitude, latitude)
        hospitals_df.at[index, 'Koordinate'] = point
    except:
        hospitals_df.at[index, 'Koordinate'] = None

In [17]:
#Display(hospitals_df)

In [18]:
# Konvertiere Pandas DataFrame in GeoPandas DataFrame
hospitals_gp_df = gpd.GeoDataFrame(hospitals_df, geometry='Koordinate', crs='EPSG:4326')

#print(hospitals_gp_df)

# Save to a GeoJSON file
output_file = "data/hospitals_saxony.geojson"
hospitals_gp_df.to_file(output_file, driver="GeoJSON")

# Creating Cities Data

In [80]:
#Einlesen der Exceldatei
cities_df=pd.read_excel('data/Gemeindedaten_prepared.xlsx') 

#manuelle Änderung: Auerbach Auerbach/Erzgebirge

In [81]:
#Filterung auf lediglich Gemeindedaten (Entfernen übergeordneter Strukturen)
cities_df= cities_df[cities_df['Satzart'] == 60].reset_index(drop=True)

In [82]:
#Filterung auf lediglich Gemeindedaten (Entfernen übergeordneter Strukturen)
cities_df= cities_df[cities_df['Land'] == 14].reset_index(drop=True)

In [83]:
# Einheitliche Konvertierung zu String und Entfernen unsichtbarer Zeichen
cities_df.columns = cities_df.columns.astype(str)
cities_df.columns = cities_df.columns.str.strip()

#Auswählen und umbenennen der relevanten Spalten
new_columns = {
	'Gemeindename':'Gemeindename',
	'Bev. insgesamt':'Bevoelkerung_insgesamt',
	'Bev. männlich':'Bevoelkerung_maennlich',
    'Bev. weiblich':'Bevoelkerung_weiblich',
	'Bev. je km2':'Bevoelkerung_je_km2',
	'Längengrad':'Längengrad',
	'Breitengrad':'Breitengrad',
}
cities_df = cities_df[list(new_columns.keys())].rename(columns=new_columns)

In [84]:
#Filterung auf lediglich Gemeindedaten (Entfernen übergeordneter Strukturen)
#cities_df= cities_df[cities_df['Kreis'] == 21].reset_index(drop=True)

In [85]:
#display(cities_df)

In [86]:
#umwandeln der Spalten Längengrad und Breitengrad in ein Point Objekt
def make_point(row):
    return Point(row['Längengrad'], row['Breitengrad'])

cities_df['Längengrad'] = cities_df['Längengrad'].str.replace(',', '.').astype(float)
cities_df['Breitengrad'] = cities_df['Breitengrad'].str.replace(',', '.').astype(float)


# Anwenden der Funktion auf den DataFrame
cities_df['coordinate_centre'] = cities_df.apply(make_point, axis=1)

In [87]:
#Korrekturen in der Gemeindename Spalte, damit Gemeinden korrekt erkannt werden
# Prüfen, ob die Spalte den String ', test' enthält und diesen entfernen
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace(', Stadt', '')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace(', Universitätsstadt', '')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace(', Hochschulstadt', '')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace(', Garnisonsstadt', '')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace(', Motorradstadt', '')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace('/Sa.', '')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace(', Kurort', '')

#spezifische Korrekturen
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace('Wurzen', 'Stadt Wurzen')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace('Reichenbach im Vogtland', 'Stadt Reichenbach im Vogtland')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace('Eibenstock', 'Stadt Eibenstock')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace('Stollberg/Erzgeb.', 'Stadt Stollberg/Erzgeb.')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace('Mulda', 'Mulda/Sa.')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace('Triebel/Vogtl.', 'Triebel')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace('Geithain', 'Stadt Geithain')
cities_df['Gemeindename'] = cities_df['Gemeindename'].str.replace('Quitzdorf am See / Kwetanecy pri jezoru', 'Quitzdorf am See')

cities_df['Gemeindename'] = cities_df['Gemeindename'] + ', Saxony, Germany'
display(cities_df)

,Gemeindename,Bevoelkerung_insgesamt,Bevoelkerung_maennlich,Bevoelkerung_weiblich,Bevoelkerung_je_km2,Längengrad,Breitengrad,coordinate_centre
0,"Chemnitz, Saxony, Germany",248563.0,122678.0,125885.0,1125.0,12.921602,50.833593,POINT (12.921602 50.833593)
1,"Amtsberg, Saxony, Germany",3666.0,1816.0,1850.0,158.0,12.987465,50.747579,POINT (12.987465 50.747579)
2,"Annaberg-Buchholz, Saxony, Germany",19382.0,9321.0,10061.0,689.0,13.003162,50.580572,POINT (13.003162 50.580572)
3,"Aue-Bad Schlema, Saxony, Germany",19864.0,9500.0,10364.0,545.0,12.700986,50.585486,POINT (12.700986 50.585486)
4,"Breitenbrunn/Erzgeb., Saxony, Germany",5084.0,2459.0,2625.0,85.0,12.768669,50.474623,POINT (12.768669 50.474623)
...,...,...,...,...,...,...,...,...
413,"Schönwölkau, Saxony, Germany",2685.0,1363.0,1322.0,54.0,12.497265,51.496502,POINT (12.497265 51.496502)
414,"Dreiheide, Saxony, Germany",2133.0,1071.0,1062.0,63.0,12.929244,51.563299,POINT (12.929244 51.563299)
415,"Torgau, Saxony, Germany",19802.0,9765.0,10037.0,193.0,13.004299,51.558563,POINT (13.004299 51.558563)
416,"Jesewitz, Saxony, Germany",3074.0,1528.0,1546.0,59.0,12.562548,51.422934,POINT (12.562548 51.422934)


In [4]:
def get_city_geometry(city_name):
    try:
        return ox.geocode_to_gdf(city_name).geometry.iloc[0]
    except:
        return None

# Geometrien für jede Stadt abrufen
cities_df['geometry'] = cities_df['Gemeindename'].apply(get_city_geometry)

# Konvertieren in ein GeoDataFrame
gdf = gpd.GeoDataFrame(cities_df, geometry='geometry')

#die Koordiante muss in wkt (well known text) umgewandelt werden, da in einem GeoJSON File nicht zwei Geo Spalten exisitieren können
gdf['coordinate_centre'] = gdf['coordinate_centre'].apply(lambda geom: geom.wkt)

gdf.drop(['Längengrad','Breitengrad'], axis=1, inplace=True)

# Überprüfen des GeoDataFrame
#print(gdf)

# Speichern als Shapefile oder GeoJSON
#gdf.to_file("data/cities_saxony_3.geojson", driver="GeoJSON") # als GeoJSON

NameError: name 'cities_df' is not defined

## Hinzufügen leipzig, Dresden Stadteile

In [89]:
Stadtteile_df=pd.read_excel('data/Stadtteile_Erweiterung.xlsx') 
Stadtteile_df['Gemeindename'] = Stadtteile_df['Gemeindename'] + ', Saxony, Germany'

In [90]:
Stadtteile_df['geometry'] = Stadtteile_df['Gemeindename'].apply(get_city_geometry)

In [91]:
Stadtteile_gdf = gpd.GeoDataFrame(Stadtteile_df, geometry='geometry')

In [92]:
Stadtteile_gdf['coordinate_centre'] = Stadtteile_gdf['geometry'].centroid
Stadtteile_gdf['coordinate_centre'] =  Stadtteile_gdf['coordinate_centre'].apply(lambda geom: geom.wkt)

In [93]:
#Stadtteile_gdf['Land'] = 14
#Stadtteile_gdf['RB'] = 7
#Stadtteile_gdf['Kreis'] = 13
Stadtteile_gdf['Bevoelkerung_maennlich'] = pd.NA
Stadtteile_gdf['Bevoelkerung_weiblich'] = pd.NA
display(Stadtteile_gdf)
gdf_combined = pd.concat([gdf, Stadtteile_gdf], ignore_index=True)
display(gdf_combined)

gdf_combined = gdf_combined[gdf_combined['Gemeindename'] != 'Leipzig, Saxony, Germany']
gdf_combined = gdf_combined[gdf_combined['Gemeindename'] != 'Dresden, Saxony, Germany']

,Gemeindename,Bevoelkerung_insgesamt,Bevoelkerung_je_km2,geometry,coordinate_centre,Bevoelkerung_maennlich,Bevoelkerung_weiblich
0,"Mitte Leipzig, Saxony, Germany",70221,13.96,"POLYGON ((12.32765 51.35960, 12.32766 51.35958...",POINT (12.371525736639791 51.340282998865874),<NA>,<NA>
1,"Nordost Leipzig, Saxony, Germany",51011,26.31,"POLYGON ((12.38876 51.35204, 12.38902 51.35193...",POINT (12.431701972902333 51.38055184580945),<NA>,<NA>
2,"Ost Leipzig, Saxony, Germany",88544,40.73,"POLYGON ((12.39257 51.34547, 12.39258 51.34546...",POINT (12.475825877874607 51.33657841259512),<NA>,<NA>
3,"Südost Leipzig, Saxony, Germany",64493,34.72,"POLYGON ((12.39258 51.33398, 12.39313 51.33360...",POINT (12.454104562472132 51.29869807599321),<NA>,<NA>
4,"Süd Leipzig, Saxony, Germany",68068,16.95,"POLYGON ((12.33978 51.30168, 12.33999 51.30155...",POINT (12.380324103022538 51.30258534854953),<NA>,<NA>
5,"Südwest Leipzig, Saxony, Germany",56817,46.56,"POLYGON ((12.24747 51.26588, 12.24875 51.26519...",POINT (12.30314528153998 51.28298875094314),<NA>,<NA>
6,"West Leipzig, Saxony, Germany",56677,14.69,"POLYGON ((12.23665 51.31978, 12.23797 51.31710...",POINT (12.272086986702295 51.31802092686017),<NA>,<NA>
7,"Altwest Leipzig, Saxony, Germany",61832,26.21,"POLYGON ((12.23767 51.37270, 12.23776 51.37245...",POINT (12.291070853507449 51.34887802896363),<NA>,<NA>
8,"Nordwest Leipzig, Saxony, Germany",36472,39.07,"POLYGON ((12.24782 51.37633, 12.24965 51.37626...",POINT (12.31070546259153 51.39069212439115),<NA>,<NA>
9,"Nord Leipzig, Saxony, Germany",74583,38.61,"POLYGON ((12.34081 51.36516, 12.34113 51.36485...",POINT (12.405247941634416 51.40260435752866),<NA>,<NA>


C:\Users\OPEF\AppData\Local\Temp\ipykernel_28308\2011459957.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gdf_combined = pd.concat([gdf, Stadtteile_gdf], ignore_index=True)


,Gemeindename,Bevoelkerung_insgesamt,Bevoelkerung_maennlich,Bevoelkerung_weiblich,Bevoelkerung_je_km2,coordinate_centre,geometry
0,"Chemnitz, Saxony, Germany",248563.0,122678.0,125885.0,1125.0,POINT (12.921602 50.833593),"POLYGON ((12.72753 50.81659, 12.72766 50.81626..."
1,"Amtsberg, Saxony, Germany",3666.0,1816.0,1850.0,158.0,POINT (12.987465 50.747579),"POLYGON ((12.96295 50.74210, 12.96354 50.74113..."
2,"Annaberg-Buchholz, Saxony, Germany",19382.0,9321.0,10061.0,689.0,POINT (13.003162 50.580572),"POLYGON ((12.97035 50.57889, 12.97052 50.57834..."
3,"Aue-Bad Schlema, Saxony, Germany",19864.0,9500.0,10364.0,545.0,POINT (12.700986 50.585486),"POLYGON ((12.62920 50.62355, 12.62925 50.62257..."
4,"Breitenbrunn/Erzgeb., Saxony, Germany",5084.0,2459.0,2625.0,85.0,POINT (12.768669 50.474623),"POLYGON ((12.69678 50.48108, 12.70285 50.46902..."
...,...,...,...,...,...,...,...
441,"Langebrück, Dresden, Saxony, Germany",3840.0,NaN,NaN,NaN,POINT (13.841557607720416 51.13335592779901),"POLYGON ((13.81637 51.12749, 13.81651 51.12744..."
442,"Schönborn, Dresden, Saxony, Germany",508.0,NaN,NaN,NaN,POINT (13.86535818986348 51.14669093325667),"POLYGON ((13.83885 51.14635, 13.84021 51.14604..."
443,"Mobschatz, Dresden, Saxony, Germany",1480.0,NaN,NaN,NaN,POINT (13.629008014418906 51.067718882813985),"POLYGON ((13.59899 51.06797, 13.59900 51.06795..."
444,"Schönfeld-Weißig, Dresden, Saxony, Germany",13046.0,NaN,NaN,NaN,POINT (13.907606215087396 51.04215557708693),"POLYGON ((13.84011 51.03656, 13.84025 51.03647..."


In [2]:
#display(Stadtteile_gdf)

In [95]:
# Speichern als Shapefile oder GeoJSON
gdf_combined.to_file("data/cities_saxony.geojson", driver="GeoJSON") # als GeoJSON

In [96]:
print(gdf_combined['Gemeindename'].nunique())

444


In [3]:
#display(gdf_combined)

# Scraping

In [1]:
#! pip install webdriver-manager
#! pip install selenium
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
import time

def get_website_content(url):
    options = Options()
    options.headless = False  # Deaktivierung des Headless-Modus
    service = Service(GeckoDriverManager().install())

    browser = webdriver.Firefox(service=service, options=options)
    
    try:
        # Öffnen der URL außerhalb der Schleife
        browser.get(url)
        
        # Erste Scroll-Operation um 3000 Pixel
        first_scroll = True
        
        while True:
            try:
                # Warten, bis ein bestimmtes Element geladen ist
                time.sleep(2)
                
                if first_scroll:
                    # Erste Scroll-Operation um 3000 Pixel
                    browser.execute_script("window.scrollBy(0, 3000);")
                    first_scroll = False
                else:
                    # Nachfolgende Scroll-Operationen um 1500 Pixel
                    browser.execute_script("window.scrollBy(0, 2400);")
                
                time.sleep(2)
                
                # Versuchen, den "Mehr laden"-Button zu finden und zu klicken
                mehr_laden_button = WebDriverWait(browser, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, '.ce-search-result-list__more-buttons__next'))
                )
                mehr_laden_button.click()

            except Exception as e:
                print(f"Fehler aufgetreten: {e}")
                break

        # Abrufen des Seiteninhalts
        content = browser.page_source
    finally:
        browser.quit()

    with open('webpage_content.txt', 'w', encoding='utf-8') as file:
        file.write(content)
    print("Webseiteninhalt wurde in 'webpage_content.txt' geschrieben.")

# Beispielverwendung
url = 'https://bundes-klinik-atlas.de/krankenhaussuche/?tx_solr%5Bgeolabel%5D=Sachsen&tx_solr%5Blatlon%5D=50.9295798%2C13.4585052&searchtype=free-search'

content = get_website_content(url)
#print(content)


ValueError: response body:
{"message":"API rate limit exceeded for 95.90.182.138. (But here's the good news: Authenticated requests get a higher rate limit. Check out the documentation for more details.)","documentation_url":"https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting"}

request url:
https://api.github.com/repos/mozilla/geckodriver/releases/latest
response headers:
{'Date': 'Mon, 22 Jul 2024 21:17:10 GMT', 'Server': 'Varnish', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'deny', 'X-XSS-Protection': '1; mode=block', 'Content-Security-Policy': "default-src 'none'; style-src 'unsafe-inline'", 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-RateLimit-Used, X-RateLimit-Resource, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-

# alter Stuff

In [79]:
# Define the place names with measures
places_with_measures = [
    "Neustadt, Dresden, Saxony, Germany, 616093",

]

# Prepare a list to store the geocoded polygons with measures
polygons = []

# Retrieve the boundary polygons for each place and add measure
for place in places_with_measures:
    parts = place.split(", ")
    place_name = ", ".join(parts[:-1])
    measure = int(parts[-1])
    polygon = ox.geocode_to_gdf(place_name)
    polygon['population'] = measure
    polygons.append(polygon)
    print(place_name)
    print(polygon.centroid)

# Combine the boundary polygons
combined_polygon = gpd.GeoDataFrame(pd.concat(polygons, ignore_index=True))

# Set the coordinate reference system to WGS84 (EPSG:4326)
combined_polygon.set_crs(epsg=4326, inplace=True)

# Save the combined polygon as a GeoJSON file
output_file = "cities_test.geojson"
combined_polygon.to_file(output_file, driver="GeoJSON")

print(f"GeoJSON file saved as {output_file}")


Neustadt, Dresden, Saxony, Germany
0    POINT (13.76258 51.07556)
dtype: geometry
GeoJSON file saved as cities_test.geojson


C:\Users\OPEF\AppData\Local\Temp\ipykernel_28308\1419818723.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  print(polygon.centroid)


In [ ]:
#print(combined_polygon)

In [72]:
'''
addresses = [
    "Eintrag 1, Brandenburger Tor, Berlin, Germany",
    "Schloss Sanssouci, Potsdam, Germany",
    "Hamburger Chaussee 22, Kiel, Germany"
]

coordinates = []
for address in addresses:
    location_point = ox.geocode(address)
    coordinates.append(location_point)

for address, coord in zip(addresses, coordinates):
    print(f"Coordinates of {address}: {coord}")
'''

'\naddresses = [\n    "Eintrag 1, Brandenburger Tor, Berlin, Germany",\n    "Schloss Sanssouci, Potsdam, Germany",\n    "Hamburger Chaussee 22, Kiel, Germany"\n]\n\ncoordinates = []\nfor address in addresses:\n    location_point = ox.geocode(address)\n    coordinates.append(location_point)\n\nfor address, coord in zip(addresses, coordinates):\n    print(f"Coordinates of {address}: {coord}")\n'

In [73]:
from shapely.geometry import Point

# Define the addresses
addresses = [
    "Ev. Diakonissenkrankenhaus Hauptstandort,249,Georg-Schwarz-Str. 49 Leipzig",
    "HELIOS-Park-Klinikum Hauptstandort,670,Strümpellstr. 41,Leipzig",
    "Herzzentrum Leipzig GmbH,440,Strümpellstr. 39,Leipzig",
    "Klinikum St. Georg GmbH,63,Nikolai-Rumjanzew-Str. 100,Leipzig",
    "Klinikum St. Georg GmbH,617,Delitzscher Str. 141,Leipzig",
    "St. Elisabeth Krankenhaus Leipzig gGmbH,442,Biedermannstr.,Leipzig",
    "Universitätsklinikum Leipzig AöR,1421,Liebigstr. 18,Leipzig",
    "Neurologisches Rehabilitationszentrum Leipzig,60,Muldentalweg 1,Bennewitz",
    "Sana-Kliniken Leipziger Land Klinikum Borna,507,Rudolf-Virchow-Str. 2,Borna",
    "Diakoniewerk Zschadraß gemeinnützige GmbH,165,Im Park 15a,Colditz",
    "Muldentalkliniken Standort Wurzen,177,Kleiststr. 5,Grimma",
    "Muldentalkliniken Standort Wurzen,178,Kutusowstr. 70,Wurzen",
    "Sana-Kliniken Leipziger Land Klinikum Borna,59,Pestalozzistr. 9,Zwenkau",
    "MediClin Waldkrankenhaus Bad Düben,125,Gustav-Adolf-Str. 15a,Bad Düben",
    "Kreiskrankenhaus Delitzsch GmbH Klinik Delitzsch,135,Dübener Str. 3-9,Delitzsch",
    "Kreiskrankenhaus Delitzsch GmbH Klinik Delitzsch,135,Wilhelm-Grune-Str. 5-8,Eilenburg",
    "COLLM KLINIK OSCHATZ GmbH,190,Parkstr. 1,Oschatz",
    "Helios Klinik Schkeuditz,150,Leipziger Str. 45,Schkeuditz",
    "Sächsisches Krankenhaus Altscherbitz - Hauptstandort,253,Leipziger Str. 59,Schkeuditz",
    "Kreiskrankenhaus Torgau Johann Kentmann,271,Christianistr. 1,Torgau",
    "Fachkrankenhaus Hubertusburg gGmbH,194,Schloss Hubertusburg 63,Wermsdorf"]

geocoded_data = []

# Geocode each address
for address in addresses:
    parts = address.split(",")

    # Ensure parts contain at least three elements: name, measure, and address
    if len(parts) < 3:
        print(f"Skipping invalid address format: {address}")
        continue
    
    name = parts[0]
    measure = int(parts[1])
    full_address = ", ".join(parts[2:])
    
    try:
        location_point = ox.geocode(full_address)
        point = Point(location_point[1], location_point[0])  # Note: Point(x, y) -> Point(longitude, latitude)
        geocoded_data.append({'name': name, 'Bettenkapazität': measure, 'address': full_address, 'geometry': point})
    except Exception as e:
        print(f"Error geocoding address '{full_address}': {e}")
        continue

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(geocoded_data)

#print(gdf)

# Set the coordinate reference system to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Save to a GeoJSON file
output_file = "hospitals.geojson"
gdf.to_file(output_file, driver="GeoJSON")

#print(f"GeoJSON file saved as {output_file}")

Error geocoding address 'Schloss Hubertusburg 63, Wermsdorf': Nominatim could not geocode query 'Schloss Hubertusburg 63, Wermsdorf'


In [74]:
print(geocoded_data)

[{'name': 'Ev. Diakonissenkrankenhaus Hauptstandort', 'Bettenkapazität': 249, 'address': 'Georg-Schwarz-Str. 49 Leipzig', 'geometry': <POINT (12.322 51.343)>}, {'name': 'HELIOS-Park-Klinikum Hauptstandort', 'Bettenkapazität': 670, 'address': 'Strümpellstr. 41, Leipzig', 'geometry': <POINT (12.435 51.308)>}, {'name': 'Herzzentrum Leipzig GmbH', 'Bettenkapazität': 440, 'address': 'Strümpellstr. 39, Leipzig', 'geometry': <POINT (12.437 51.307)>}, {'name': 'Klinikum St. Georg GmbH', 'Bettenkapazität': 63, 'address': 'Nikolai-Rumjanzew-Str. 100, Leipzig', 'geometry': <POINT (12.297 51.319)>}, {'name': 'Klinikum St. Georg GmbH', 'Bettenkapazität': 617, 'address': 'Delitzscher Str. 141, Leipzig', 'geometry': <POINT (12.376 51.384)>}, {'name': 'St. Elisabeth Krankenhaus Leipzig gGmbH', 'Bettenkapazität': 442, 'address': 'Biedermannstr., Leipzig', 'geometry': <POINT (12.375 51.309)>}, {'name': 'Universitätsklinikum Leipzig AöR', 'Bettenkapazität': 1421, 'address': 'Liebigstr. 18, Leipzig', 'geo

In [19]:
import osmnx as ox
from shapely.geometry import Polygon, Point
import geopandas as gpd

def get_district_center(city_name, district_name):
    # Suchen und laden Sie die Geometrien des Stadtteils
    city = ox.geocode_to_gdf(city_name)
    district = city[city['name'] == district_name]

    if district.empty:
        print(f"District {district_name} not found in {city_name}")
        return None, None

    # Extrahieren der Geometrie des Stadtteils
    district_polygon = district.geometry.values[0]

    # Berechnen des Mittelpunkts der Geometrie
    center = district_polygon.centroid

    return center.y, center.x  # Rückgabe der Latitude und Longitude des Mittelpunkts

# Beispiel-Nutzung
city_name = "Leipzig"
district_name = "Mitte"
center_lat, center_lon = get_district_center(city_name, district_name)

if center_lat is not None and center_lon is not None:
    print(f"Center of {district_name}: Latitude {center_lat}, Longitude {center_lon}")
else:
    print(f"No center found for {district_name} in {city_name}")


District Mitte not found in Leipzig
No center found for Mitte in Leipzig
